In [43]:
import sys
# sys.path.append("/Users/bubble/Desktop/Project/Infrasound Sensor/Layout/Code/acousticsensor")

import gdsfactory as gf
# TODO
# 1. Change one side of the cell_temp to be comparation with the width
# 2. Different with the width and same width with gap

cell_temp = gf.Component()

In [44]:
# bottom left
L = [250, 500, 750, 1000]
g = [50, 75, 100]
width = [[314, 310, 410], [522, 618, 614], [834, 772, 818], [1042, 1080, 1022]]
w = 2
t = 0.5
beamNum = []
side =[None] * 4
k = (len(L) * len(g))
mesh = [None] * k
for i in range(k):
    mesh[i] = gf.Component()
for i in range(4):
    side[i] = gf.Component()
block = gf.Component()
count = 0
origin = (5000 / 4, 0)

for i in range(4):
    for j in range(3):
        # single grid
        row =   L[i] // (g[j]+w)+1
        column = row if row % 2 == 0 else row + 1
        beamNum.append((row+1)+ column)
        height = L[i] / row - w
        sg = gf.components.rectangle(size=(g[j], height), layer=(8, 0))
        for m in range(column):
            for n in range(row):
                sf_ref = mesh[count] << sg
                sf_ref.move((origin[0]+m*(g[j]+w), origin[1]+n*(height+w)))
        # frame
        fra_width = column * (g[j] + w) + w
        frame = gf.components.rectangle(size=(fra_width, L[i]), layer=(9, 0))
        frame_ref = mesh[count] << frame
        frame_ref.movex(origin[0]-w)
        # crystal
        mid = origin[0] - w + fra_width / 2
        cry_size = 32
        # cry_height = height+0.1 if j != 0 else 2*height+w+0.1
        crystal = gf.components.rectangle(size=(cry_size, cry_size), layer=(10, 0))
        crystal_ref = mesh[count] << crystal
        crystal_ref.move((mid-cry_size/2, L[i]-cry_size-w+0.05))
        # deposition area
        gold_size = cry_size - 6
        gold = gf.components.rectangle(size=(gold_size, gold_size+2), layer=(11, 0))
        gold_ref = mesh[count] << gold
        gold_ref.move((mid-gold_size/2, L[i]-gold_size-w-3))
        # move the mesh
        temp = count // 3
        mesh_ref = side[temp] << mesh[count]
        mesh_ref.movex((count%3) * 5000 / 4 - L[i] / 2) 
        count += 1
    # length mark
    T = gf.components.text(f"{L[i]}{width[i]}", size=50, layer=(1, 0))
    T_ref = side[i] << T
    T_ref.move((2500, -150))

for i in range(len(side)):
    side_ref = block << side[i]
    if i == 0:
        pass
    elif i == 1:
        side_ref.dmirror_y(5000/2).dmirror_x(5000/2)
    elif i == 2:
        side_ref.drotate(angle=-90, center=(5000/2, 5000/2))
    else:
        side_ref.drotate(angle=90, center=(5000/2, 5000/2))

# side.show()
# print(beamNum)



In [45]:
# backside etching 5mm * 5mm (5743.44um)
backside = gf.Component()
size = 5743.44
diff = (size - 5000) / 2
for i in range(4):
    backside_temp = gf.components.rectangle(size=(size, size), layer=(3, 0))
    backside_ref = backside << backside_temp
    if i == 0:
        backside_ref.move((-diff, -diff))
    elif i == 1:
        backside_ref.move((10000-diff, -diff))
    elif i == 2:
        backside_ref.move((-diff, 10000-diff))
    else:
        backside_ref.move((10000-diff, 10000-diff))
# backside.show()

In [46]:
# structure for each die
# structure for 4 sides
fblock = gf.Component()
for i in range(4):
    if i == 0:
        block_ref = fblock << block
    elif i == 1:
        block_ref = fblock << block
        block_ref.move((10000, 0))
    elif i == 2:
        block_ref = fblock << block
        block_ref.move((10000, 10000))
    else:
        block_ref = fblock << block
        block_ref.move((0, 10000))

In [ ]:
# order
order = gf.Component()
text_array = []
for i in range(4):
    text_array.append(gf.Component())

for i in range(4):
    T = gf.components.text(f"MSL{i+1}", size=20, layer=(1, 0))
    for j in range(4):
        order_ref = text_array[i] << T
        if j == 0:
            order_ref.move((-80, 0))
        elif j == 1:
            order_ref.move((-80, 5000))
        elif j == 2:
            order_ref.move((5020, 0))
        else:
            order_ref.move((5020, 5000))
    text_array_ref = order << text_array[i]
    if i == 0:
        pass
    elif i == 1:
        text_array_ref.move((10000, 0))
    elif i == 2:
        text_array_ref.move((0, 10000))
    else:
        text_array_ref.move((10000, 10000))


In [48]:
# backside frame 5mm * 5mm 
backframe = gf.Component()
size = 5000
for i in range(4):
    backframe_temp = gf.components.rectangle(size=(size, size), layer=(3, 0))
    backframe_ref = backframe << backframe_temp
    if i == 0:
        backframe_ref.move((0, 0))
    elif i == 1:
        backframe_ref.move((10000, 0))
    elif i == 2:
        backframe_ref.move((0, 10000))
    else:
        backframe_ref.move((10000, 10000))
# backside.show()

In [ ]:
# boolean operation
# do backside etching - frame
outside = gf.boolean(A = backframe, B = fblock,  operation="not", layer1=(3, 0), layer2=(9, 0), layer=(2, 0))
outside = gf.boolean(A = outside, B = fblock,  operation="not", layer1=(2, 0), layer2=(10, 0), layer=(1, 0))

cell_temp << outside

#  do holes etching - crystal
holes = gf.boolean(A = fblock, B = fblock,  operation="not", layer1=(8, 0), layer2=(10, 0), layer=(1, 0))
cell_temp << holes
# add holes to cell_temp
pc = gf.boolean(A = fblock, B = fblock,  operation="not", layer1=(11, 0), layer2=(30, 0), layer=(2, 0))
cell_temp << pc
cell_temp << backside
# add marker/order
marker = gf.boolean(A = fblock, B = fblock,  operation="not", layer1=(1, 0), layer2=(15, 0), layer=(1, 0))
cell_temp << marker
cell_temp << order
# frame
frame1 = gf.components.rectangle(size=(15000, 15000), layer=(20, 0))
frame2 = gf.components.rectangle(size=(20000, 20000), layer=(21, 0))
frame2_ref = cell_temp << frame2
frame2_ref.move((-2500, -2500))
cell_temp << frame1

cell_mesh_large = gf.Component()
cell_ref = cell_mesh_large << cell_temp
cell_ref.move((2500, -7500))
# cell_mesh_large.show()
# cell_mesh_large.write_gds("mesh.gds")
# cell_mesh_large.plot()